In [1]:
#To run:
#ipython simple_nn.py --pylab -i

import pandas as pd
import numpy as np
import datetime

import torch
from torch.autograd import Variable

In [2]:
#load data
# df_merged_volume = pd.read_pickle("phase1_training_vol_route_weather_joined_table.pkl")

df_merged_volume = pd.read_csv('../data/preprocessed_input.csv')

In [3]:
df_merged_volume.columns

,datetime,dayofweek,hour,dt0_10_total_vol,dt20_10_total_vol,dt40_10_total_vol,dt60_10_total_vol,dt80_10_total_vol,dt100_10_total_vol,dt120_10_total_vol,...,dt0_rel_humidity,dt0_precipitation,dt180_pressure,dt180_sea_pressure,dt180_wind_direction,dt180_wind_speed,dt180_temperature,dt180_rel_humidity,dt180_precipitation,is_holiday
0,2016-09-20 00:00:00,1,0,14,13,7,6,5,5,6,...,68.0,0.0,1013.5,1018.5,354.0,2.6,23.9,58.0,0.0,0
1,2016-09-20 03:00:00,1,3,10,8,5,11,8,15,9,...,58.0,0.0,1012.2,1017.2,7.0,4.6,26.9,43.0,0.0,0
2,2016-09-20 06:00:00,1,6,13,20,29,38,30,31,53,...,43.0,0.0,1012.6,1017.6,11.0,4.5,25.4,42.0,0.0,0
3,2016-09-20 09:00:00,1,9,51,68,59,56,60,63,57,...,42.0,0.0,1013.7,1018.7,25.0,1.7,22.1,53.0,0.0,0
4,2016-09-20 12:00:00,1,12,53,62,75,98,82,84,81,...,53.0,0.0,1014.0,1019.0,326.0,2.2,20.5,65.0,0.0,0
5,2016-09-20 15:00:00,1,15,71,64,73,48,57,48,38,...,65.0,0.0,1013.8,1018.8,327.0,1.4,19.4,74.0,0.0,0
6,2016-09-20 18:00:00,1,18,26,29,20,17,29,21,12,...,74.0,0.0,1014.1,1019.1,330.0,1.0,18.6,80.0,0.0,0
7,2016-09-20 21:00:00,1,21,17,29,37,30,17,19,20,...,80.0,0.0,1015.5,1020.5,219.0,0.7,21.7,69.0,0.0,0
8,2016-09-21 00:00:00,2,0,12,6,7,7,4,6,6,...,69.0,0.0,1015.4,1020.4,17.0,2.4,25.2,59.0,0.0,0
9,2016-09-21 03:00:00,2,3,5,6,10,16,14,11,10,...,59.0,0.0,1013.1,1018.1,328.0,3.0,26.2,56.0,0.0,0


In [5]:
'''
5 Checkpoint: 1-0 , 1-1, 2-0, 3-0, 3-1
    2nd index{ 0: entry, 1: exit }
    
6 Paths: A-2, A-3, B-1, B-3, C-1, C-3

Row Time Duration capture for Volume and Travel time is 4 hrs [before 2hrs and after 2 hrs] ]


Features:
1. Volume (20m)
-
2. ETC (20m)
3. Motorcycle (20m)
4. Cargocar (20m)
5. Privatecar (20m)
6. Unknowncar (20m)
-
7. routetime median (20m)
-
8. pressure (3 hrs)
9. sea_pressure (3 hrs)
10. wind_direction (3 hrs)
11. wind_speed (3 hrs)
12. temperature (3 hrs)
13. rel_humidity (3 hrs)
14. precipitation (3 hrs)

== TODO:
1. Moving avg for 3hr's features
2. Investigate Backfill for FillNA
3. Create the Input Tensor (3D)


== Procedures:
Preprocessing
0. 
1. Backfill for FillNA
2. Moving avg for 3hr's features
3. Select the relevant row (time duration, 6am and 3pm)
4. Normalization
5. Create the Input Tensor (3D)

Build NN:
Keras
'''

for i in df_merged_volume.columns:
    print(i)

datetime
dayofweek
hour
dt0_10_total_vol
dt20_10_total_vol
dt40_10_total_vol
dt60_10_total_vol
dt80_10_total_vol
dt100_10_total_vol
dt120_10_total_vol
dt140_10_total_vol
dt160_10_total_vol
dt180_10_total_vol
dt200_10_total_vol
dt220_10_total_vol
dt0_11_total_vol
dt20_11_total_vol
dt40_11_total_vol
dt60_11_total_vol
dt80_11_total_vol
dt100_11_total_vol
dt120_11_total_vol
dt140_11_total_vol
dt160_11_total_vol
dt180_11_total_vol
dt200_11_total_vol
dt220_11_total_vol
dt0_20_total_vol
dt20_20_total_vol
dt40_20_total_vol
dt60_20_total_vol
dt80_20_total_vol
dt100_20_total_vol
dt120_20_total_vol
dt140_20_total_vol
dt160_20_total_vol
dt180_20_total_vol
dt200_20_total_vol
dt220_20_total_vol
dt0_30_total_vol
dt20_30_total_vol
dt40_30_total_vol
dt60_30_total_vol
dt80_30_total_vol
dt100_30_total_vol
dt120_30_total_vol
dt140_30_total_vol
dt160_30_total_vol
dt180_30_total_vol
dt200_30_total_vol
dt220_30_total_vol
dt0_31_total_vol
dt20_31_total_vol
dt40_31_total_vol
dt60_31_total_vol
dt80_31_total_vol

In [6]:
# only select relevant time, 6am (4 hrs info) and 3pm (4 hrs info)
sel_rows = df_merged_volume[ lambda df : (df.hour==6) | (df.hour==15)]

In [7]:
sel_rows

,datetime,dayofweek,hour,dt0_10_total_vol,dt20_10_total_vol,dt40_10_total_vol,dt60_10_total_vol,dt80_10_total_vol,dt100_10_total_vol,dt120_10_total_vol,...,dt0_rel_humidity,dt0_precipitation,dt180_pressure,dt180_sea_pressure,dt180_wind_direction,dt180_wind_speed,dt180_temperature,dt180_rel_humidity,dt180_precipitation,is_holiday
2,2016-09-20 06:00:00,1,6,13,20,29,38,30,31,53,...,43.0,0.0,1012.6,1017.6,11.0,4.5,25.4,42.0,0.0,0
5,2016-09-20 15:00:00,1,15,71,64,73,48,57,48,38,...,65.0,0.0,1013.8,1018.8,327.0,1.4,19.4,74.0,0.0,0
10,2016-09-21 06:00:00,2,6,36,41,75,78,84,95,122,...,56.0,0.0,1013.2,1018.2,348.0,1.5,24.9,67.0,0.0,0
13,2016-09-21 15:00:00,2,15,48,46,49,46,45,46,47,...,93.0,0.0,1013.1,1018.1,341.0,1.7,20.2,94.0,0.0,0
18,2016-09-22 06:00:00,3,6,6,10,36,28,40,34,50,...,54.0,0.0,1010.6,1015.6,91.0,2.6,26.8,57.0,0.0,0
21,2016-09-22 15:00:00,3,15,61,48,57,58,52,42,29,...,80.0,0.0,1011.6,1016.5,33.0,1.1,21.9,84.0,0.0,0
26,2016-09-23 06:00:00,4,6,12,17,32,28,41,42,49,...,53.0,0.0,1008.2,1013.2,96.0,4.5,27.2,61.0,0.0,0
29,2016-09-23 15:00:00,4,15,47,66,78,76,86,44,45,...,79.0,0.0,1008.8,1013.7,317.0,1.8,21.4,88.0,0.0,0
34,2016-09-24 06:00:00,5,6,8,19,24,28,33,35,43,...,55.0,0.0,1006.3,1011.2,78.0,5.2,27.8,64.0,0.0,0
37,2016-09-24 15:00:00,5,15,55,63,66,81,59,48,73,...,82.0,0.0,1007.1,1012.0,325.0,1.4,21.9,91.0,0.0,0


In [8]:
#replace NA entries and normalize data
sel_rows_mean = sel_rows.mean(0)
sel_rows_std = sel_rows.std(0)

In [10]:
sel_rows_norm = sel_rows.fillna(sel_rows_mean)
sel_rows_norm = (sel_rows_norm - sel_rows_mean) / sel_rows_std

/Users/Kin/anaconda/lib/python3.5/site-packages/pandas/core/ops.py:995: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1 but corresponding boolean dimension is 56
  yrav = yrav[mask]


In [11]:
len(sel_rows_norm)

56

In [ ]:
#split data to train and test set
np.random.seed(1234)
mask = np.random.rand(len(sel_rows_norm)) < 0.75
train_rows = sel_rows_norm[mask]
test_rows = sel_rows_norm[~mask]

#select useful feature input cols and output cols
vols = [(1,0), (1,1), (2,0), (3,0), (3,1)]
routes = [ ("A",2), ("A",3), ("B",1), ("B",3), ("C",1), ("C",3) ]

inCols = []
for dt in range(0,120,20):
  for (tid,io) in vols:
    inCols.append('dt%i_%s%s_%s_vol' %(dt,tid,io,'total') )
  for (p,q) in routes:
    inCols.append('dt%i_%s%s_routetime_median'%(dt,p,q) )

outCols = []
for dt in range(120,240,20):
  for (tid,io) in vols:
    outCols.append('dt%i_%s%s_%s_vol' %(dt,tid,io,'total') )
  for (p,q) in routes:
    outCols.append('dt%i_%s%s_routetime_median'%(dt,p,q) )

train_input = train_rows[inCols]
train_ans = train_rows[outCols]

test_input = test_rows[inCols]
test_ans = test_rows[outCols]

inDim = len(inCols)
outDim = len(outCols)

#-----------------------------------
#pytorch part
#-----------------------------------
# Create random Tensors to hold inputs and outputs, and wrap them in Variables.
x = Variable(torch.FloatTensor(train_input.as_matrix()), requires_grad=False)
y = Variable(torch.FloatTensor(train_ans.as_matrix()  ), requires_grad=False)

x_test = Variable(torch.FloatTensor(test_input.as_matrix()), requires_grad=False)
y_test = Variable(torch.FloatTensor(test_ans.as_matrix()  ), requires_grad=False)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(inDim, 10),
    torch.nn.ReLU(),
    torch.nn.Linear(10, 10),
    torch.nn.ReLU(),
    torch.nn.Linear(10, outDim),
)
loss_fn = torch.nn.MSELoss(size_average=False)

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Variables it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

loss_seq = []
loss_test_seq = []

for t in range(3000):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)
    y_pred_test = model(x_test)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    loss_test = loss_fn(y_pred_test, y_test)
    print(t, loss.data[0], loss_test.data[0])

    loss_seq.append( loss.data[0] )
    loss_test_seq.append( loss_test.data[0] )

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable weights
    # of the model)
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

plot(loss_seq)
plot(loss_test_seq)

print min(loss_test_seq) , loss_test_seq.index(min(loss_test_seq))